In [9]:
# !pip install langdetect googletrans==4.0.0-rc1

In [8]:
import pandas as pd
from langdetect import detect
from googletrans import Translator, LANGUAGES

ModuleNotFoundError: No module named 'langdetect'

In [ ]:
df = pd.read_csv("../Data_Sets/electronics_reviews.csv")

In [ ]:
df.head(2)

# Using Google translator

In [ ]:
# Initialize the translator
translator = Translator()

def detect_and_translate(text):
    try:
        # Detect language
        lang = detect(text)
        if lang != 'en':
            # Translate to English
            translated = translator.translate(text, src=lang, dest='en').text
            return translated
        return text
    except Exception as e:
        return str(e)

# Apply the detect_and_translate function to the text column
print(df['title'][:5].apply(detect_and_translate))

In [ ]:
print(df['text'][:5].apply(detect_and_translate))

# Using open AI

In [24]:
# !pip install --upgrade httpx openai

In [81]:
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = 'sk-proj-qPZr4Y5io2BHaP2LmDPlT3BlbkFJnkgNHZ3O1RMkPlyG5VcU' # from sajjan

# Sample dataset
data = {
    'text': [
        'Hello, how are you?',
        'Bonjour, comment ça va?',
        'Hola, ¿cómo estás?',
        'Hallo, wie geht es dir?',
        'Ciao, come stai?',
        'Buen vendedor'
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

def detect_and_translate(text):
    try:
        # Use OpenAI API to determine the language and translate
        response = openai.Completion.create(
            model="gpt-3.5-turbo",
            prompt=f"Detect the language and translate this text to English: '{text}'",
            max_tokens=100
        )
        translated_text = response.choices[0].text.strip()
        return translated_text
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return text

# Apply the detect_and_translate function to the text column
df['text'][:5].apply(detect_and_translate)

Error processing text: Hello, how are you?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text: Bonjour, comment ça va?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text: Hola, ¿cómo estás?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text: Hallo, wie geht es dir?. Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error processing text

0        Hello, how are you?
1    Bonjour, comment ça va?
2         Hola, ¿cómo estás?
3    Hallo, wie geht es dir?
4           Ciao, come stai?
Name: text, dtype: object

# Named Entity recognition using matcher

In [51]:
import pandas as pd
import spacy
from spacy.matcher import Matcher

In [54]:
# !python -m spacy download en_core_web_sm

In [55]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

In [89]:
# Sample dataset
# data = {
#     'review': [
#         'The phone has a great battery life, excellent camera, and loud sound.',
#         'I love the bluetooth connectivity of this speaker.',
#         'This laptop has an amazing display and the keyboard is very comfortable.',
#         'The smartwatch has good battery but the GPS is not accurate.',
#         'The headphones have great sound quality and the battery lasts long.'
#     ]
# }

# # Create a DataFrame
# df = pd.DataFrame(data)

# Define product features to look for
product_features = [
    "battery", "camera", "sound", "bluetooth", "display", "keyboard", "GPS", "headphones"
]

# Create a spaCy Matcher instance
matcher = Matcher(nlp.vocab)

# Add patterns for each product feature
for feature in product_features:
    pattern = [{"LOWER": feature.lower()}]
    matcher.add(feature, [pattern])

def extract_features(review):
    doc = nlp(review) 
    matches = matcher(doc)
    extracted_features = [doc[start:end].text for match_id, start, end in matches]
    return extracted_features

# Apply the extract_features function to the review column
df['text'][:50].apply(extract_features)

0                      []
1     [Sound, headphones]
2                      []
3                      []
4                 [Sound]
5                      []
6               [display]
7                      []
8                      []
9                      []
10                     []
11                     []
12                     []
13                     []
14                     []
15                     []
16                     []
17                     []
18                     []
19                     []
20                     []
21                     []
22                     []
23                     []
24                     []
25                     []
26                     []
27                     []
28                     []
29                     []
30               [camera]
31                     []
32                     []
33                     []
34         [sound, sound]
35                     []
36                     []
37                     []
38          

In [92]:
import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")


# Define keywords related to product features
product_keywords = {
    'phone': ['battery', 'camera', 'sound', 'bluetooth'],
    'speaker': ['bluetooth', 'sound', 'connectivity'],
    'laptop': ['display', 'keyboard', 'battery'],
    'smartwatch': ['battery', 'GPS'],
    'headphones': ['sound', 'battery']
}

def extract_product_features(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Initialize an empty list to store extracted features
    features = []

    # Iterate over entities and tokens in the document
    for ent in doc.ents:
        # Check if the entity label is related to products or organizations
        if ent.label_ in ['PRODUCT', 'ORG']:
            # Check if the entity text is a product keyword
            if ent.text.lower() in product_keywords:
                features.extend(product_keywords[ent.text.lower()])

    # Iterate over tokens in the document
    for token in doc:
        # Check if the token is a noun and not a stop word
        if token.pos_ == 'NOUN' and not token.is_stop:
            # Check if the token text is a product keyword
            for product, keywords in product_keywords.items():
                if token.text.lower() in keywords:
                    features.append(token.text)
                    break

    return list(set(features))  # Return unique features

# Apply the extract_product_features function to the review column
df['text'][:50].apply(extract_product_features)

0            []
1            []
2            []
3            []
4            []
5            []
6     [display]
7            []
8            []
9            []
10           []
11           []
12           []
13           []
14           []
15           []
16           []
17           []
18           []
19           []
20           []
21           []
22           []
23           []
24           []
25           []
26           []
27           []
28           []
29           []
30     [camera]
31           []
32           []
33           []
34      [sound]
35           []
36           []
37           []
38           []
39           []
40           []
41           []
42           []
43           []
44           []
45           []
46      [sound]
47           []
48           []
49    [battery]
Name: text, dtype: object

In [96]:
# nltk.download('punkt')

In [113]:
import nltk

lines = df["text"][5]
print(lines)
# function to test if something is a noun
is_noun = lambda pos: pos[:2] == 'NN'
# do the nlp stuff
tokenized = nltk.word_tokenize(lines)
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 

print(nouns)

I LOVE my Amazon 4K Fire Stick!!!  My TV is only 5 years old but already out of date!  I couldn’t load any new apps but now I do it all on my Fire Stick, and I have only had a brief pause in streaming once, most likely due to my wifi connection.  I like it so much I have purchased a second one for one of my other TVs and I have recommended it to so many friends who were having the same issues I was with their relatively new, yet “old” TV.  Since I already have an echo dot in my room, I just tell Alexa what I want on the TV and she does it!  Definitely one of my best purchases for 2020!!!!
['Amazon', 'Fire', 'Stick', 'TV', 'years', 'date', 't', 'load', 'apps', 'Fire', 'Stick', 'pause', 'connection', 'TVs', 'friends', 'issues', 'TV', 'dot', 'room', 'Alexa', 'TV', 'purchases']


In [117]:
import openai

# Your OpenAI API key
api_key = 'sk-proj-qPZr4Y5io2BHaP2LmDPlT3BlbkFJnkgNHZ3O1RMkPlyG5VcU'

# Review text
review_text = """
I LOVE my iPhone 12 Pro Max!!! The camera quality is incredible, the battery life lasts all day, 
and the performance is blazing fast. I upgraded from an older model and couldn't be happier. 
Definitely worth the investment!
"""

# OpenAI GPT-3 prompt
prompt = f"Extract the product name and top 3 features from the following review:\n{review_text}\nProduct:"

# Call OpenAI GPT-3 API
response = openai.Completion.create(
    engine="GPT-3.5",  # or use "text-davinci-003" for GPT-3.5
    prompt=prompt,
    max_tokens=150,  # Adjust as necessary
    api_key=api_key
)

# Parse the response to extract product name and features
output = response.choices[0].text.strip().split('\n')
product_name = output[0]
features = [feat.strip('- ') for feat in output[1:4]]  # Extract top 3 features

print(f"Product: {product_name}")
print("Top 3 features:")
for feature in features:
    print(f"- {feature}")


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [123]:
# !pip install transformers

In [130]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("describeai/gemini-small")
model = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini-small")

ImportError: cannot import name 'AutoTokenizer' from 'transformers' (unknown location)

In [159]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="describeai/gemini-small")

ImportError: cannot import name 'pipeline' from 'transformers' (unknown location)

In [133]:
# !pip install yake

In [158]:
import yake
kw_extractor = yake.KeywordExtractor()
text = """I have spent several years carrying an Inreach and it is hard to believe how unreliable this device is. Updating firmware is a nightmare. Syncing, nightmare. User Interface is clunky. Getting a computer to both use the sync/update software and actually see the device plugged in via USB is a nightmare. You can either go with a less robust satellite beacon or go full satellite phone - either is a more reliable alternative."""
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 10
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('carrying an Inreach', 0.012849913978126995)
('spent several years', 0.031243910890458804)
('years carrying', 0.031243910890458804)
('Inreach', 0.07288751237548687)
('nightmare', 0.07934818494128172)
('unreliable this device', 0.1705844289713273)
('spent', 0.17406120936370134)
('years', 0.17406120936370134)
('carrying', 0.17406120936370134)
('hard', 0.17406120936370134)


In [169]:
!pip install summa

DEPRECATION: Loading egg at /Users/quicktech/anaconda3/lib/python3.11/site-packages/huggingface_hub-0.22.0rc0-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=ad87ddba2544482a9e35cfc01530e00b09be73b0e2b3af5aaaf034fcd218ba57
  Stored in directory: /Users/quicktech/Library/Caches/pip/wheels/10/2d/7a/abce87c4ea233f8dcca0d99b740ac0257eced1f99a124a0e1f
Successfully built summa


In [172]:
from summa import keywords
text = """I LOVE my Amazon 4K Fire Stick!!!  My TV is only 5 years old but already out of date!  I couldn‚Äôt load any new apps but now I do it all on my Fire Stick, and I have only had a brief pause in streaming once, most likely due to my wifi connection.  I like it so much I have purchased a second one for one of my other TVs and I have recommended it to so many friends who were having the same issues I was with their relatively new, yet ‚Äúold‚Äù TV.  Since I already have an echo dot in my room, I just tell Alexa what I want on the TV and she does it!  Definitely one of my best purchases for 2020!!!!"""
TR_keywords = keywords.keywords(text, scores=True)
print(TR_keywords[0:10])

[('new', 0.2982219516463651), ('k', 0.20433726316510273)]


In [177]:
!pip install keybert

DEPRECATION: Loading egg at /Users/quicktech/anaconda3/lib/python3.11/site-packages/huggingface_hub-0.22.0rc0-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
  Obtaining dependency information for huggingface-hub>=0.15.1 from https://files.pythonhosted.org/packages/69/d6/73f9d1b7c4da5f0544bc17680d0fa9932445423b90cd38e1ee77d001a4f5/huggingface_hub-0.23.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.4
    Uninstalling huggingface-hub-0.23.4:
      Successfully uninstalled huggingface-hub-0.23.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ludwig 0.9.3 requires fsspec[htt

In [176]:
from keybert import KeyBERT

ImportError: cannot import name 'AutoConfig' from 'transformers' (unknown location)

In [178]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [190]:
raw_text="I LOVE my Amazon 4K Fire Stick!!!  My TV is only 5 years old but already out of date!  I couldn‚Äôt load any new apps but now I do it all on my Fire Stick, and I have only had a brief pause in streaming once, most likely due to my wifi connection.  I like it so much I have purchased a second one for one of my other TVs and I have recommended it to so many friends who were having the same issues I was with their relatively new, yet ‚Äúold‚Äù TV.  Since I already have an echo dot in my room, I just tell Alexa what I want on the TV and she does it!  Definitely one of my best purchases for 2020!!!!"

text1= NER(raw_text)

# Now, we print the data on the NEs found in this text sample.

for word in text1.ents:
    print(word.text,word.label_)
displacy.render(text1,style="ent",jupyter=True)    

LOVE ORG
Amazon 4K Fire Stick ORG
only 5 years old DATE
Fire Stick FAC
second ORDINAL
one CARDINAL
one CARDINAL
‚ PERSON
Alexa ORG
2020 DATE


In [189]:
raw_text="I have spent several years carrying an Inreach and it is hard to believe how unreliable this device is. Updating firmware is a nightmare. Syncing, nightmare. User Interface is clunky. Getting a computer to both use the sync/update software and actually see the device plugged in via USB is a nightmare. You can either go with a less robust satellite beacon or go full satellite phone - either is a more reliable alternative."


text1= NER(raw_text)

# Now, we print the data on the NEs found in this text sample.

for word in text1.ents:
    print(word.text,word.label_)
displacy.render(text1,style="ent",jupyter=True)    

several years DATE
Inreach PERSON
Interface ORG
USB ORG


In [188]:
raw_text="Sound is good quality. I like it that although I only bought one set of headphones the charger has an option for a second set incase I want to buy another one."


text1= NER(raw_text)

# Now, we print the data on the NEs found in this text sample.

for word in text1.ents:
    print(word.text,word.label_)
displacy.render(text1,style="ent",jupyter=True)    

second ORDINAL
